# Components, mixtures and phase equilibrium model basics

Phasepy unit for temperature is [K], unit of pressure is [bar] and unit of molar volume is [cm^3/mol].

In [1]:
import numpy as np
from phasepy import mixture, component, virialgamma, preos

## Single components

Physical properties are specified when a component is created. Typical properties include critical properties (``Tc``, ``Pc``, ``Zc``, ``Vc``), acentric factor (``w``), Antoine Coefficients (``Ant``) and group specification for Modified UNIFAC group contribution (``GC``) method.

In [2]:
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1})

print('Water saturation pressure at 373 K:', water.psat(T=373.0), 'bar')
print('Water liquid molar volume at 310 K:', water.vlrackett(T=310.0), 'cm3/mol')
print('Ethanol saturation pressure at 373 K:', ethanol.psat(T=373.0), 'bar')
print('Ethanol liquid molar volume at 310 K:', ethanol.vlrackett(T=310.0), 'cm3/mol')

Water saturation pressure at 373 K: 1.0072796747419537 bar
Water liquid molar volume at 310 K: 16.46025809309672 cm3/mol
Ethanol saturation pressure at 373 K: 2.233335305328437 bar
Ethanol liquid molar volume at 310 K: 56.32856995891473 cm3/mol


In [3]:
# Example 1. Peng Robinson EOS for single component (without volume translation)
eos = preos(ethanol)
print('Saturation pressure, liquid and vapor molar volumes:', eos.psat(T=350.0))
print('Liquid density:', eos.density(T=350.0, P=1.0, state='L'))
print('Vapor density:', eos.density(T=350.0, P=1.0, state='V'))

Saturation pressure, liquid and vapor molar volumes: (array([0.98800647]), array([66.75754804]), array([28799.31921623]))
Liquid density: 0.01497960198094922
Vapor density: 3.515440899573752e-05


In [4]:
# Example 2. Peng Robinson EOS for single component (with volume translation, parameter c)
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    c=5.35490936, Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1})
eos = preos(ethanol, volume_translation=True)
print('Saturation pressure, liquid and vapor molar volumes:', eos.psat(T=350.0))
print('Liquid density:', eos.density(T=350.0, P=1.0, state='L'))
print('Vapor density:', eos.density(T=350.0, P=1.0, state='V'))

Saturation pressure, liquid and vapor molar volumes: (array([0.98800647]), array([61.40263868]), array([28793.96430687]))
Liquid density: 0.01628597159790686
Vapor density: 3.5161028012629526e-05


## Mixtures

Mixtures are created from components with the ``mixture`` class function.
The mixture function allows setting interaction parameters, such for NRTL model (``mixture.NRTL``), Wilson model (``mixture.wilson``), Redlich-Kister polynomial (``mixture.rk``) and UNIFAC model (``mixture.unifac``).

In [5]:
mix = mixture(ethanol, water)
print('Component names:', mix.names)
print('Number of components:', mix.nc)
print('Component saturation pressures:', mix.psat(T=373.0))
print('Component liquid molar volumes:', mix.vlrackett(T=310.0))

Component names: ['ethanol', 'water']
Number of components: 2
Component saturation pressures: [2.23333531 1.00727967]
Component liquid molar volumes: [56.32856996 16.46025809]


## Phase equilibrium models

Phase equilibrium models are created from a mixture and model specifications. Model specific parameters must be provided to the mixture.

In [6]:
# Example 3. Initialization of NRTL activity coefficient model for liquid, and Abbott virial model for vapor
alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
g = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
g1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, g, g1)
eos = virialgamma(mix, virialmodel='Abbott', actmodel='nrtl')

In [7]:
# Example 4. Inititalization of Redlich Kister expansion for liquid, Tsonopoulos virial model for vapor
C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = virialgamma(mix, virialmodel='Tsonopoulos', actmodel='rk')

In [8]:
# Example 5. Inititalization of Dortmund Modified UNIFAC for liquid, ideal gas law for vapor
mix.unifac()
eos = virialgamma(mix, virialmodel='ideal_gas', actmodel='unifac')

In [9]:
# Example 6. Initialization of Peng Robinson EOS for mixture with Quadratic mixing rule.
Kij = np.array([[0, -0.11], [-0.11, 0]])
mix.kij_cubic(Kij)
eos = preos(mix, mixrule='qmr')

In [10]:
# Example 7. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with NRTL.
alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
g = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
g1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, g, g1)
eos = preos(mix, mixrule='mhv_nrtl')

In [11]:
# Example 8. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with Modified UNIFAC.
mix.unifac()
eos = preos(mix, mixrule='mhv_unifac')

In [12]:
# Example 9. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with Redlich Kister.
C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = preos(mix, mixrule='mhv_rk')

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```